# Outline experiment final assignment

## Presentation of a story

1. Multiple pages of text on screen. Participants can view a page as long as they want, and press the spacebar to proceed to the next page.
2. Text and background will be in grayscale color. It will be selected at random what the contrast between text and background will be.
3. Contrast between text and background per participant will have to be stored.
4. Reading time per page is collected.
5. filenames have to be dependend on participant name

## Questionnaires about reading experiences

1. Story World Absorption Scale (SWAS) and Appreciation Questionnaire Adjectives (AQA) will be presented.
2. Participants will have to answer each question using a slider (7 possible positions of the slider, by default the slider is in the middle, at position 4.
3. Output from the qustionnaires will be collected.

## Analysis
1. Calculate mean per subscale for SWAS
2. Do PCA and calculate factor scores for AQA
3. Calculate total reading time per participant
4. Calculate correlation between SWAS & AQA & total reading time
5. Analyze whether means for SWAS subscales, factor scores for AQA, and total reading time is related to level of contrast between text and background.

## Visualization
1. Visualize correlation SWAS & AQA % total reading time
2. Visualize Mean per subscale SWAS per level of contrast
3. Visualize factor score AQA per level of contrast
4. Visualize total reading time per level of contrast

# Step 1
## Presentation of a story

1. Multiple pages of text on screen. Participants can view a page as long as they want, and press the spacebar to proceed to the next page.
2. Text and background will be in grayscale color. It will be selected at random what the contrast between text and background will be.
3. Contrast between text and background per participant will have to be stored.
4. Reading time per page is collected.
5. filenames have to be dependend on participant name

In [2]:
import subprocess as sp
sp.run(['python', 'storypresentation.py'])

CompletedProcess(args=['python', 'storypresentation.py'], returncode=0)

In [47]:
import csv

pageList = [pagenum for pagenum in range(4)]

textperpage = ["Er was eens",
               "Een meisje",
              "en zij at graag",
              "een ijsje"]

with open(r'readingtime.tsv', 'w') as readingfile:
        writer = csv.writer(readingfile, delimiter = '\t')
        writer.writerow(['Page','Time'])

for page in pageList:
    print(textperpage[page])
    with open(r'readingtime.tsv', 'a') as readingfile:
        writer = csv.writer(readingfile, delimiter='\t')
        writer.writerow([page,'tba'])

Er was eens
Een meisje
en zij at graag
een ijsje
